In [1]:
import requests
import folium
import numpy as np
import pandas as pd
import json

# L'URL de l'API pour donné de compteur 
limit_datacount = 100 # télécharger 100 données mais ne pas saugarder localement 
offset_datacount = 0
dateEtHeure ='2023-12-24T23'
#api avec un filtre "order by" dans l'ordre décroissant de dates dans datacount
api_url_datacount = "https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?order_by=date%20DESC&limit=" + str(limit_datacount) +"&offset=" + str(offset_datacount)

#L'URL de l'API pour localisation des sites de comptage
api_url_localisation = "https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-sites/records?limit=29"

#L'URL de l'API pour télécharger toutes les données 
api_url_export = "https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/exports/json?lang=fr&timezone=Europe%2FBerlin"


# #local_data telecharge depuis serveur
local_data = pd.read_json("local_data.json")

update_data = [] # initialise update_data par une liste

dateEtHeure = local_data.iloc[0]['date'][:-12] # pour avoir la forme de variable dateEtHeure identique que celle declarée dessus

#L'URL de l'API pour la mise à jour de données (where date > dateEtHeure le plus récent indiqué dans local_data, order by date descendant, timezone Europe/Berlin)
api_url_update_data ="https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?where=date%3Edate'"+dateEtHeure+"%3A00%3A00%2B01%3A00'&order_by=date%20DESC&limit="+str(limit_datacount)+"&offset="+str(offset_datacount)+"&timezone=Europe%2FBerlin"


# Effectuer une requête GET pour obtenir les données
response_updateData = requests.get(api_url_update_data) # dictionnaire (indice total_count, indice results)
response_localisation = requests.get(api_url_localisation)

if (response_updateData.status_code != 200):
   
    print(f'Échec de la requête HTTP, code de statut {response_updateData.status_code}')

if (response_localisation.status_code != 200):
   
    print(f'Échec de la requête HTTP, code de statut {response_localisation.status_code}')



# Si la requête a réussi (code de statut HTTP 200), vous pouvez accéder aux données
data_localisation = response_localisation.json()

total_count = response_updateData.json()['total_count']

print(total_count)

if (total_count != 0) : # le nombre de données mises à jour 
    while (total_count >= 100) : # data limit is 100
        update_data.extend(requests.get(api_url_update_data).json()['results'])
        total_count -= 100
        offset_datacount +=  100
        api_url_update_data ="https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?where=date%3Edate'"+dateEtHeure+"%3A00%3A00%2B01%3A00'&order_by=date%20DESC&limit="+str(limit_datacount)+"&offset="+str(offset_datacount)+"&timezone=Europe%2FBerlin"

    limit_datacount = total_count
    api_url_update_data ="https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records?where=date%3Edate'"+dateEtHeure+"%3A00%3A00%2B01%3A00'&order_by=date%20DESC&limit="+str(limit_datacount)+"&offset="+str(offset_datacount)+"&timezone=Europe%2FBerlin"
    update_data.extend(requests.get(api_url_update_data).json()['results'])
    list_local_data =local_data.to_dict(orient='records')
    update_data.extend(list_local_data)
    # Utilisez json.dump() pour sauvegarder le dictionnaire dans le fichier JSON

    with open('local_data.json', 'w') as fichier_json:
        json.dump(update_data, fichier_json)


# df_update_data =pd.DataFrame(update_data)
# df_combined = pd.concat([df_update_data,local_data])


# results_datacount = local_data['results']
#results_localisation = data_localisation['results']



    



    

# print("ID:", first_result['id'])
# print("Date:", first_result['date'])
# print("Counts:", first_result['counts'])
# print("Status:", first_result['status'])
# print("Name:", first_result['name'])
# print("User Type:", first_result['user_type'])
# print("Photo URL:", first_result['photourl'])
# print("Date_j:", first_result['date_j'])


# for i in range (len(results_localisation)):
    
#     lon = results_localisation[i]['coordinates']['lon']
#     lat = results_localisation[i]['coordinates']['lat']
#     if i==0:
#         map_center = [lat, lon]
#         mymap = folium.Map(location=map_center, zoom_start=15)
#     coordinates = [lat, lon]
   
#     # Dessiner un cercle proportionnel à l'intervalle
#     radius = np.sqrt(results_localisation[i]['interval'] / np.pi)   # Ajustez le facteur selon vos besoins
#     folium.CircleMarker(location=coordinates, radius=radius, color='blue', fill=True).add_to(mymap)
#     # Afficher la carte
# mymap.save('map.html')



0


In [ ]:
import pandas as pd
import plotly.express as px
import ipywidgets as ipw
import matplotlib.pyplot as plt
df = pd.read_json('local_data.json')
df
df['date_j'].unique()

In [ ]:
mois = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
an = ['2022', '2023', '2024']
date_jour = df['date_j'].unique()

# Convertir en datetime
df['date'] =  pd.to_datetime(df['date'], utc=True)
df['date_j'] = pd.to_datetime(df['date_j'])
df['month_year'] = df['date_j'].dt.to_period('M')

# Créer une liste de tous les mois de janvier 2022 à janvier 2024
months = pd.period_range(start='2022-01', end='2024-01', freq='M')

# Créer un DataFrame avec une colonne 'mois' contenant les mois et une colonne 'mean_count_per_day' initialisée à NaN
df_result = pd.DataFrame({'mois': months, 'mean_count_per_day': [None] * len(months)})

# Remplir la colonne 'mean_count_per_day' avec les valeurs moyennes appropriées
for i, month_period in enumerate(months):
    # Calculer la valeur moyenne pour le mois donné
    total_counts = df[df['month_year'] == month_period]['counts'].sum()
    mean_counts_per_day = total_counts / month_period.to_timestamp().days_in_month / len(df['name'].unique())
    # Assigner la valeur moyenne à la ligne correspondante dans le DataFrame résultant
    df_result.at[i, 'mean_count_per_day'] = mean_counts_per_day

plt.figure(figsize=(10, 6))
df_result['mois_str'] = df_result['mois'].astype(str)

# Tracer un graphique linéaire
plt.figure(figsize=(10, 6))
plt.plot(df_result['mois_str'], df_result['mean_count_per_day'], marker='o', linestyle='-')

plt.title('Passages moyens journaliers au mois')
plt.xlabel('Mois')
plt.ylabel('Passages journaliers moyens ')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Créer une liste de tous les heures
heures = pd.period_range(start='00:00', end='23:00', freq='h')
heures_only = [str(heure.strftime('%H:%M:%S')) for heure in heures]

# Créer une liste de tous les jours de la semaine
days_in_a_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Créer un DataFrame avec une colonne 'day' contenant les jours, une colonne 'hour' contenant les heures, 
# et une colonne 'mean_count_per_day' initialisée à NaN
df_result = pd.DataFrame({'day': days_in_a_week * len(heures_only), 
                          'hour': heures_only * len(days_in_a_week), 
                          'mean_count_per_day': [None] * len(days_in_a_week) * len(heures_only)})

# Regrouper les données par heure et sommer les valeurs de la colonne "counts" pour chaque heure
df['day_name'] = df['date_j'].dt.day_name()

# Extraire uniquement l'heure de la colonne 'date' et stocker le résultat dans une nouvelle colonne 'heure'
df['heure'] = df['date'].dt.strftime('%H:%M:%S')

# Remplir la colonne 'mean_count_per_day' avec les valeurs moyennes appropriées
for hour in heures_only:
    for day_name in days_in_a_week:
        # Filtrer les données pour le jour et l'heure donnés
        filtered_data = df[(df['day_name'] == day_name) & (df['heure'] == hour)]
        # Calculer la moyenne des valeurs de la colonne 'counts' pour ces données filtrées
        mean_count = filtered_data['counts'].mean()
        # Assigner la valeur moyenne à la ligne correspondante dans le DataFrame résultant
        df_result.loc[(df_result['day'] == day_name) & (df_result['hour'] == hour), 'mean_count_per_day'] = round(mean_count)

print(df_result)

In [ ]:
# Convertir les colonnes 'day' et 'hour' en variables catégorielles avec un ordre spécifique
df_result['day'] = pd.Categorical(df_result['day'], categories=days_in_a_week, ordered=True)
df_result['hour'] = pd.Categorical(df_result['hour'], categories=heures_only, ordered=True)
df_result['mean_count_per_day'] = pd.to_numeric(df_result['mean_count_per_day'], errors='coerce')
# Pivoter les données pour avoir les jours comme index, les heures comme colonnes et les valeurs de 'mean_count_per_day' comme valeurs
heatmap_data = df_result.pivot(index='hour', columns='day', values='mean_count_per_day')



# Tracer le heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt=".2f", cbar_kws={'label': 'Mean Count per Day'})
plt.title('Heatmap of Mean Count per Day')
plt.xlabel('Hour')
plt.ylabel('Day')
plt.tight_layout()
plt.show()

In [ ]:
# Convertir les colonnes 'day' et 'hour' en variables catégorielles avec un ordre spécifique
df_result['day'] = pd.Categorical(df_result['day'], categories=days_in_a_week, ordered=True)
df_result['hour'] = pd.Categorical(df_result['hour'], categories=heures_only, ordered=True)

# Pivoter les données pour avoir les jours comme index, les heures comme colonnes et les valeurs de 'mean_count_per_day' comme valeurs
heatmap_data = df_result.pivot(index='day', columns='hour', values='mean_count_per_day')

# Tracer le heatmap avec Plotly Express
fig = px.imshow(heatmap_data, x=heures_only, y=days_in_a_week, color_continuous_scale='viridis')
fig.update_layout(title='Heatmap of Mean Count per Day', xaxis_title='Hour', yaxis_title='Day')

# Créer une interface interactive avec ipywidgets pour afficher le heatmap
def update_heatmap(color_scale):
    fig.update_traces(colorscale=color_scale)

color_scale_dropdown = ipw.Dropdown(options=['viridis', 'plasma', 'inferno'], value='viridis', description='Color Scale:')
ipw.interactive(update_heatmap, color_scale=color_scale_dropdown)